```{raw} html
<div id="qe-notebook-header" align="right" style="text-align:right;">
        <a href="https://quantecon.org/" title="quantecon.org">
                <img style="width:250px;display:inline;" width="250px" src="https://assets.quantecon.org/img/qe-menubar-logo.svg" alt="QuantEcon">
        </a>
</div>
```

# Heavy-Tailed Distributions

```{admonition} New website
:class: warning

We have moved this lecture to a new lecture series.

See [An Undergraduate Lecture Series for the Foundations of Computational Economics](https://intro.quantecon.org)
```